# Power Flow Neural Network Training with Pandapower

This notebook demonstrates how to create a pipeline for training a neural network to learn power flow solutions using pandapower's internal states.

In [1]:
import pandapower as pp
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.tensorboard import SummaryWriter # for pytorch visualization

from sklearn.preprocessing import StandardScaler # normalize input features and target values
from sklearn.model_selection import ParameterGrid # for hyperparameter tuning

## 1. Create Dataset Class

First, we create a PyTorch dataset class that interfaces with pandapower:

In [2]:
class PowerFlowDataset(Dataset):
    def __init__(self, base_network, num_samples=1000):
        """
        Initialize the dataset with a base network and number of samples.
        
        Parameters:
        base_network (pandapowerNet): The base pandapower network.
        num_samples (int): Number of samples to generate.
        """
        self.base_net = base_network
        self.num_samples = num_samples
        self.samples = []
        self.scaler_input = StandardScaler()
        self.scaler_output = StandardScaler()
        self.generate_samples()
        self.normalize_samples()
        
    def generate_samples(self):
        """
        Generate samples by randomly modifying the base network and running power flow.
        """
        for _ in range(self.num_samples):
            net = self.base_net.deepcopy()
            # Modify the network randomly to generate different samples
            # For example, change load values, generator outputs, etc.
            pp.runpp(net)
            Ybus = net._ppc["internal"]["Ybus"].toarray()
            S = net._ppc["internal"]["Sbus"]
            V_mag = net.res_bus.vm_pu.values
            V_ang = net.res_bus.va_degree.values
            self.samples.append({
                "input": np.concatenate([Ybus.real.flatten(), 
                                         Ybus.imag.flatten(),
                                         S.real, 
                                         S.imag]),
                "output": np.concatenate([V_mag, V_ang])
            })

    
    def normalize_samples(self):
        """
        Normalize the input features and target values.
        """
        inputs = np.array([sample["input"] for sample in self.samples])
        outputs = np.array([sample["output"] for sample in self.samples])
        self.scaler_input.fit(inputs)
        self.scaler_output.fit(outputs)
        for sample in self.samples:
            sample["input"] = self.scaler_input.transform([sample["input"]])[0]
            sample["output"] = self.scaler_output.transform([sample["output"]])[0]
    
    def __len__(self):
        return self.num_samples
    
    def __getitem__(self, idx):
        sample = self.samples[idx]
        return {
            'input': torch.FloatTensor(sample['input']),
            'output': torch.FloatTensor(sample['output'])
        }

## 2. Create Neural Network Model

Next, we define our neural network architecture:

In [3]:
class PowerFlowDNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(PowerFlowDNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

## 3. Training Pipeline

Create the training function:

In [4]:
def train_power_flow_model(base_network, num_epochs=100, batch_size=32):
    # Create dataset
    dataset = PowerFlowDataset(base_network)
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # Initialize model, loss function, and optimizer
    input_size = len(dataset[0]['input'])
    output_size = len(dataset[0]['output'])
    model = PowerFlowDNN(input_size=input_size, hidden_size=512, output_size=output_size)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # Initialize TensorBoard writer
    writer = SummaryWriter()

    # Training loop
    for epoch in range(num_epochs):
        model.train()
        for batch in train_loader:
            batch_inputs = batch['input']
            batch_targets = batch['output']

            # Forward pass
            outputs = model(batch_inputs)
            loss = criterion(outputs, batch_targets)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Validation loop
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for batch in val_loader:
                batch_inputs = batch['input']
                batch_targets = batch['output']
                outputs = model(batch_inputs)
                loss = criterion(outputs, batch_targets)
                val_loss += loss.item()
        val_loss /= len(val_loader)

        # Log training and validation loss to TensorBoard
        writer.add_scalar('Loss/train', loss.item(), epoch)
        writer.add_scalar('Loss/val', val_loss, epoch)

        if epoch % 10 == 0:
            print(f"Epoch {epoch}, Loss: {loss.item():.6f}, Val Loss: {val_loss:.6f}")

    writer.close()
    return model

## 4. Usage Example

Here's how to use the trained model:

In [9]:
def predict_power_flow(model, net):
    # Run power flow to ensure internal data is available
    pp.runpp(net, calculate_voltage_angles=True)
    
    # Prepare input
    Ybus = net._ppc["internal"]["Ybus"].toarray()
    S = net._ppc["internal"]["Sbus"]
    input_tensor = torch.FloatTensor(np.concatenate([
        Ybus.real.flatten(), 
        Ybus.imag.flatten(),
        S.real, 
        S.imag
    ]))
    
    # Get prediction
    with torch.no_grad():
        output = model(input_tensor)
    
    # Split prediction into voltage magnitudes and angles
    n_buses = len(net.bus)
    V_mag_pred = output[:n_buses].numpy()
    V_ang_pred = output[n_buses:].numpy()
    
    # Get reference values from the network
    V_mag_ref = net.res_bus.vm_pu.values
    V_ang_ref = net.res_bus.va_degree.values
    
    return V_mag_pred, V_ang_pred, V_mag_ref, V_ang_ref

## 5. Example Usage

Here's how to put it all together:

In [ ]:
# Create a simple test network
# net = pp.create_empty_network()
# Add your network elements here...
net = pp.networks.example_simple()

# Train the model
model = train_power_flow_model(net)

# Make predictions
V_mag_pred, V_ang_pred, V_mag_ref, V_ang_ref = predict_power_flow(model, net)
print("Predicted voltage magnitudes:", V_mag_pred)
print("Predicted voltage angles:", V_ang_pred)
print("Reference voltage magnitudes:", V_mag_ref)
print("Reference voltage angles:", V_ang_ref)

## 6. Additional Hyperparameter Tuning


In [ ]:
def evaluate_model(model, val_loader, criterion):
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            batch_inputs = batch['input']
            batch_targets = batch['output']
            outputs = model(batch_inputs)
            loss = criterion(outputs, batch_targets)
            val_loss += loss.item()
    val_loss /= len(val_loader)
    return val_loss

def hyperparameter_tuning(base_network, param_grid):
    best_model = None
    best_loss = float('inf')
    dataset = PowerFlowDataset(base_network)
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    _, val_dataset = random_split(dataset, [train_size, val_size])
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

    for params in ParameterGrid(param_grid):
        print(f"Training with parameters: {params}")
        model = train_power_flow_model(base_network, num_epochs=params['num_epochs'], batch_size=params['batch_size'])
        val_loss = evaluate_model(model, val_loader, nn.MSELoss())
        if val_loss < best_loss:
            best_loss = val_loss
            best_model = model
    return best_model

base_network = pp.networks.example_simple()
param_grid = {
    'num_epochs': [50, 100],
    'batch_size': [16, 32],
    'hidden_size': [256, 512]
}
best_model = hyperparameter_tuning(base_network, param_grid)
# ...